In [51]:
import polars as pl
import altair as alt
from pathlib import Path
import datetime as dt

alt.data_transformers.enable("vegafusion")
data_path = Path("../data/processed")


In [52]:
### Loading the data from paper


montly_temperature = pl.read_parquet(data_path / "monthly_t2m_stats_2000_2025.parquet")


In [53]:
montly_temperature

latitude,longitude,year,month,t2m_mean_c,t2m_anomaly_c
f64,f64,i16,i8,f32,f32
90.0,-180.0,2000,1,-23.630356,0.336376
90.0,-179.75,2000,1,-23.630356,0.336376
90.0,-179.5,2000,1,-23.630356,0.336376
90.0,-179.25,2000,1,-23.630356,0.336376
90.0,-179.0,2000,1,-23.630356,0.336376
…,…,…,…,…,…
-90.0,178.75,2025,9,-50.734772,3.263893
-90.0,179.0,2025,9,-50.734772,3.263893
-90.0,179.25,2025,9,-50.734772,3.263893


In [54]:
# First graph: time series of temperature with minimum and maximums
# Aggregate temperature data (300M entries) to year-month
# AI: Asked in polars AI how to aggregate while extracting statistics
montly_temp_agg = montly_temperature.group_by("year", "month").agg(
    pl.median("t2m_mean_c").alias("t2m_median_c_avg"),
    pl.mean("t2m_mean_c").alias("t2m_mean_c_avg"),
    pl.max("t2m_mean_c").alias("t2m_max_c"),
    pl.min("t2m_mean_c").alias("t2m_min_c"),
    pl.mean("t2m_anomaly_c").alias("t2_mean_anomaly"),
    pl.max("t2m_anomaly_c").alias("t2_max_anomaly"),
    pl.len().alias("count")
    )

#adding date column to data
montly_temp_agg = montly_temp_agg.with_columns(
    pl.date(pl.col("year"), pl.col("month"), 1).alias("date")
)

In [56]:
# First graph
def temp_time_series (df):
    """
    Time series graph with min, mean and max temperature
    """
    chart = alt.Chart(df)
    base = chart.encode(x=alt.X("date:T", title="Date"))
    # I asked AI how to add the legend and use the two series efficiently, suggested transform_calculate
    line_median = (
        base.mark_line()
        .transform_calculate(series='"Median"')
        .encode(
            y=alt.Y("t2m_median_c_avg:Q", title="Temperature (°C)"),
            color=alt.Color(
                "series:N",
                scale=alt.Scale(domain=["Median","Max"], range=["#637079","#d43939"]),
                legend=alt.Legend(title="Series")
            )
        )
    )
    line_max = (
        base.mark_line()
        .transform_calculate(series='"Max"')
        .encode(y="t2m_max_c:Q", color="series:N")
    )

    return alt.layer(line_median, line_max).properties(width=800, height=300)

temp_time_series(montly_temp_agg)


alt.LayerChart(...)

In [57]:
## GRaph 2, long term anomalies last 5 years:
def plot_anomalies(df):
    """
    Line chart of temperature anomalies over time
    """
    
    chart = alt.Chart(df)
    base = chart.encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("t2_mean_anomaly:Q", title="Temperature Anomaly (°C)")
        )
    
    # Split the time series color by period
    line_pre = (
        base.transform_filter(alt.datum.year < 2020)
            .mark_line(color="#a2957cb2")
    )
    
    line_post = (
        base.transform_filter(alt.datum.year >= 2020)
                .mark_line(color="#f6a800f0")
                .encode(y="t2_mean_anomaly:Q")
        )
    
    # AI: I asked AI to help me figure out how to add average lines with date data
    # to not include the trends after (suggested the darum approach)
    cut = dt.date(2020, 1, 1)
    min_pre = df.filter(pl.col("year") < 2020).select(pl.min("date")).to_series()[0]
    max_post = df.select(pl.max("date")).to_series()[0]
    
    mean_pre = float(df.filter(pl.col("year") < 2020)["t2_mean_anomaly"].mean())
    mean_post = float(df.filter(pl.col("year") >= 2020)["t2_mean_anomaly"].mean())
    
    # To add the trend lines before and after 2020
    rule_pre_data = [{"start": min_pre, "end": cut, "mean": float(mean_pre)}]
    rule_post_data = [{"start": cut, "end": max_post, "mean": float(mean_post)}]

    rule_pre = base.mark_rule(color="#6c6b6681", strokeDash=[4, 2], strokeWidth=2).encode(
        x=alt.datum(min_pre),
        x2=alt.datum(cut),
        y=alt.datum(mean_pre),
        )
    rule_post = base.mark_rule(color="#a67c00", strokeDash=[4, 2], strokeWidth=2).encode(
        x=alt.datum(cut),
        x2=alt.datum(max_post),
        y=alt.datum(mean_post),
        )
            
    return alt.layer(
        line_pre,line_post, rule_pre, rule_post).properties(
            width=800, height=300, title="Monthly Temperature Anomalies (2000-2024)")

plot_anomalies(montly_temp_agg)


alt.LayerChart(...)

In [58]:
# Graph 3: heatmap on anomalies over year month

def anomalies_heatmap(df):

    chart = alt.Chart(df)
    # AI: Asked how to change schema color to other option and
    # Used this https://altair-viz.github.io/user_guide/generated/core/altair.Scale.html to see options
    heat_map = chart.mark_rect().encode(
        alt.X("month(date):O", title="Month"),
        alt.Y("year:O", title="Year"),
        alt.Color("mean(t2_mean_anomaly):Q", title="Avg. Temp Anomaly (°C)",
                  scale = alt.Scale(domainMid=0,
                                    range=["#175585", "#f7f7f7", "#d7191c"]))
    ).properties(width=800, height=300)
    
    return heat_map

anomalies_heatmap(montly_temp_agg)
    

alt.Chart(...)

In [59]:
# Graph 4:
# Anomalies map on 2024
def anomalies_map(df):
    df = df.filter(pl.col("year") == 2024)
    
    chart = alt.Chart(df).mark_rect().encode(
        x=alt.X("longitude:Q", title="Longitude", bin=alt.Bin(maxbins=100)),
        y=alt.Y("latitude:Q", title="Latitude", bin=alt.Bin(maxbins=100)),
        color=alt.Color(
            "mean(t2m_anomaly_c):Q",
            title="Avg Anomaly (°C)",
            scale=alt.Scale(
                domainMid=0,
                range=["#175585", "#f7f7f7", "#d7191c"]
            )
        ),
    )
    
    countries = alt.topo_feature(
        "https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json",
        "countries",
    )
    background = (
        alt.Chart(countries)
            .mark_geoshape(fill="#f3f4f600", stroke="#151414", strokeWidth=0.5)
            )
    
    
    return (chart + background).project(
        type="equirectangular").properties(
        width=800, height=400, title = "Avg. Anomaly temperature map for 2024")

anomalies_map(montly_temperature)

alt.LayerChart(...)

In [60]:
# Graph 5: Clusters where the Survey was carried out
clusters_path = data_path / "Countries-Cluser.csv"
cluster_data = pl.read_csv(clusters_path)

def bar_graph_clusters_by_country(df):
    """
    Bar graph of number of clusters by country
    """
    chart = alt.Chart(df)
    bar = chart.mark_bar(color="#d43a3abb").encode(
        y=alt.Y("country_str:N", title="Country").sort("-x"),
        x=alt.X("count():Q", title="Number of Clusters in Sample")
    ).properties(width=800, height=300)
    
    return bar

bar_graph_clusters_by_country(cluster_data)


alt.Chart(...)

In [61]:
# Graph 6
# mapping codes to countires
COUNTRY_ID_MAP = {
    "Comoros": 174,
    "Nigeria": 566,
    "Laos": 418,
    "Malawi": 454,
    "Madagascar": 450,
    "Sierra Leone": 694,
    "Georgia": 268,
    "State of Palestine": 275,
    "Gambia": 270,
    "Eswatini": 748,
    "Fiji": 242,
    "Kosovo": 383
}

def points_map(df: pl.DataFrame, country: str):
    # Optional country filter
    if country:
        df_plot = df.filter(pl.col("country_str") == country)
    else:
        df_plot = df
    # Avoid missing coordinates
    df_plot = df_plot.filter(
        pl.col("latitude").is_not_null() & pl.col("longitude").is_not_null()
    )
    # Background: countries
    # AI: Searchd AI to see how to import the map at 50 boundary resolution
    countries = alt.topo_feature(
        "https://cdn.jsdelivr.net/npm/world-atlas@2/countries-50m.json",
        "countries",
    )
    cid = COUNTRY_ID_MAP.get(country, None)
    
    
    background = (
        alt.Chart(countries)
            .transform_filter(alt.datum.id == str(cid))
            .mark_geoshape(fill="#f3f4f6", stroke="#959090", strokeWidth=0.5)
            )

    # Points
    points = (
        alt.Chart(df_plot)
        .mark_circle(size=35, color="#b41f1f62", opacity=0.8)
        .encode(
            longitude="longitude:Q",
            latitude="latitude:Q",
            tooltip=[
                alt.Tooltip("country_str:N", title="Country"),
                alt.Tooltip("longitude:Q", format=".4f"),
                alt.Tooltip("latitude:Q", format=".4f"),
            ],
        )
    )
    
    title = f"Number of Clusters in {country}" if country else "Clusters"
    
    return (background+points).project(type="mercator").properties(
        width=800, height=300, title=title)

points_map(cluster_data, "Nigeria")


alt.LayerChart(...)

In [62]:
points_map(cluster_data, "Laos")


alt.LayerChart(...)

In [63]:
points_map(cluster_data, "Malawi")

alt.LayerChart(...)

In [64]:
# Graph 7
import pandas as pd # Choosing pandas for more stata compatibility
# With survey + match data 
MICS_merge_temp = pd.read_stata(data_path/"data_NGP_Dec2024.dta")

# Filtering variables
list_variables = ['t_from_birth_to_interview',
                  't_6m_before_interv',
                  'country']
MICS_merge_temp_filtered = MICS_merge_temp[list_variables]

#MICS_TEMP_pl = pl.from_pandas(MICS_merge_temp)

def boxplot_temp_per_country(df, variable):
    """
    Boxplot of temperature distribution per country with overlaid points
    """
    # Boxplot
    box = alt.Chart(df).mark_boxplot(size=30, color = "#39b8d49f").encode(
        x=alt.X(f'{variable}:Q', title='Temperature (°C)'),
        y=alt.Y('country:N', title='Country')
    )
    
    return (box ).properties(
        width=800, 
        height=300, 
        title="Avg. Temperature Exposition for children from date of birth by Country"
    )

boxplot_temp_per_country(MICS_merge_temp_filtered, 't_from_birth_to_interview')


alt.Chart(...)

In [65]:
# Graph 8
# ECDI components (bars proportions) by wealth quintiles
# Select the ECDI variables we want
# ASked AI to help me create quintiles witn 
ecdi_vars = ['ecdi_track', 'ecdi_se_track', 'ecdi_physical_track', 'ecdi_learning_track', 'high_stim']
MICS_merge_temp['wscore'] = pd.to_numeric(MICS_merge_temp['wscore'], errors='coerce')

# Then we create quintiles
MICS_merge_temp['wscore_iqr'] = (
    MICS_merge_temp
    .groupby(['country', 'year'])['wscore']
    .transform(lambda x: pd.qcut(x, q=5, labels=False, duplicates='drop') + 1)
)

# Create a clean dataset with only "on track" (value = 1)
MICS_clean = MICS_merge_temp[ecdi_vars + ['wscore_iqr']].copy()

# Filter for only "on track" cases (value = 1.0)
for var in ecdi_vars:
    MICS_clean = MICS_clean[MICS_clean[var].notna()]

# Reshape from wide to long format using pd.melt
MICS_long = pd.melt(
    MICS_clean[MICS_clean[ecdi_vars].eq(1.0).any(axis=1)],
    id_vars=['wscore_iqr'],
    value_vars=ecdi_vars,
    var_name='ECDI_Component',
    value_name='On_Track'
)

# Keep only rows where On_Track = 1
MICS_long = MICS_long[MICS_long['On_Track'] == 1.0]

# Convert to strings for Altair
MICS_long['wscore_iqr'] = MICS_long['wscore_iqr'].astype(str)

# Clean up ECDI component names for display
label_map = {
    'ecdi_track': 'Overall ECDI',
    'ecdi_se_track': 'Socio-Emotional',
    'ecdi_physical_track': 'Physical',
    'ecdi_learning_track': 'Learning',
    'high_stim': 'High Stimulation'
}
MICS_long['ECDI_Component'] = MICS_long['ECDI_Component'].map(label_map)



def stacked_bar_by_wealth(df):
    
    quintile_colors = ["#d7191cd1", "#fdf861da", "#bfffc9a9", "#3ea0c0c5", "#0d4876"]
    
    # Create stacked bar chart
    chart = alt.Chart(df).mark_bar().encode(
        y=alt.Y('ECDI_Component:N', 
                title='ECDI Component',
                sort=['Overall ECDI', 'Socio-Emotional', 'Physical', 'Learning', 'High Stimulation']),
        x=alt.X('count():Q', 
                title='Percentage',
                stack='normalize',
                axis=alt.Axis(format='%')),
        color=alt.Color(
            'wscore_iqr:N',
            title='Wealth Quintile',
            scale=alt.Scale(
                domain=['1', '2', '3', '4', '5'],
                range=quintile_colors
            ),

        ),
    ).properties(
        width=700,
        height=300,
        title='Distribution of Wealth Quintiles Across ECDI Components (On Track Only)'
    )
    
    return chart

stacked_bar_by_wealth(MICS_long)

/var/folders/m8/p0y07mnx0xj8nxd8l9zj7fn00000gn/T/ipykernel_27165/972902683.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['country', 'year'])['wscore']
/var/folders/m8/p0y07mnx0xj8nxd8l9zj7fn00000gn/T/ipykernel_27165/972902683.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  MICS_merge_temp['wscore_iqr'] = (


alt.Chart(...)

In [67]:
import statsmodels.api as sm
import numpy as np
# Graph 09
# probability of having high stimulatiuon against temperature

# Clean data: remove missing values
MICS_clean = MICS_merge_temp[['high_stim', 't_from_birth_to_interview']].dropna()

# Prepare data for probit model
y = MICS_clean['high_stim']
X = MICS_clean['t_from_birth_to_interview']
X = sm.add_constant(X)  # Add intercept

# Fit probit model
model = sm.Probit(y, X)
results = model.fit(disp=0)

# Create prediction data
temp_range = np.linspace(
    MICS_clean['t_from_birth_to_interview'].min(),
    MICS_clean['t_from_birth_to_interview'].max(),
    100
)
X_pred = sm.add_constant(temp_range)
predicted_prob = results.predict(X_pred)

# Get standard errors for confidence interval
# AI: Asked how to calculate confidence intervals for probit predictions
predictions = results.get_prediction(X_pred)
pred_summary = predictions.summary_frame(alpha=0.05)

#Create prediction dataframe for Altair
pred_df = pd.DataFrame({
    'temperature': temp_range,
    'predicted_prob': predicted_prob,
    'lower_ci': pred_summary['ci_lower'],
    'upper_ci': pred_summary['ci_upper']
})

def scatter_stimulation_temperature(df):

    # Scatter plot of actual data
    band = alt.Chart(df).mark_area(
        opacity=0.3,
        color="#f02a0774"
    ).encode(
        x=alt.X('temperature:Q', title='Temperature Exposure (°C)'),
        y=alt.Y('lower_ci:Q', title='Probability of High Stimulation'),
        y2='upper_ci:Q'
    )
    
    # Regression line (predicted probabilities)
    line = alt.Chart(pred_df).mark_line(
        color="#2b2e41fa",
        strokeWidth=2
    ).encode(
        x=alt.X('temperature:Q'),
        y=alt.Y('predicted_prob:Q', title='Probability of High Stimulation')
    )
    
    return (band + line).properties(
        width=800,
        height=300,
        title='Probability of High Stimulation vs Temperature Exposure (Probit Model)'
    )

scatter_stimulation_temperature(pred_df)

alt.LayerChart(...)